In [11]:
del(socket_Pi)

Terminate Pi Socket Connection!


In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
# receive cmds from PC through socket
# from testComms.SocketInterface_Pi_v3 import SocketInterface_Pi
# import os
# os.chdir("/home/pi/")
from testComms.SocketInterface_Pi_v3 import SocketInterface_Pi
socket_Pi = SocketInterface_Pi(HOST_IP="192.168.31.52", HOST_PORT=50000)  # server

Starting socket: TCP...
TCP server listen @ 192.168.31.52:50005!


OSError: [Errno 99] Cannot assign requested address

In [3]:
cmd_recv = socket_Pi.subscriber_cmd()
print("Pi receive:",cmd_recv)


Pi receive: {'ly': 0.0, 'lx': 0.0, 'rx': 0.0, 'ry': 0.0, 'R1': 0, 'L1': 1, 'dpady': 0.0, 'dpadx': 0.0, 'x': 0, 'message_rate': 50}


In [4]:
from src.State import State
state = State()

command = socket_Pi.get_command(state, cmd_recv)

activate_event: True


In [ ]:
import numpy as np
import time
from src.IMU import IMU
from src.Controller import Controller
# from src.JoystickInterface import JoystickInterface
from testComms.SocketInterface_Pi_v3 import SocketInterface_Pi
from src.State import State
from pupper.HardwareInterface import HardwareInterface
from pupper.Config import Configuration
from pupper.Kinematics import four_legs_inverse_kinematics

def robot(use_imu=False):
    """Main program
    """

    # Create config
    config = Configuration()
    hardware_interface = HardwareInterface()
    # print("hardware interface established")
    # Create imu handle
    if use_imu:
        imu = IMU(port="/dev/ttyACM0")
        imu.flush_buffer()

    # Create controller and user input handles
    controller = Controller(
        config,
        four_legs_inverse_kinematics,
    )
    state = State()
    # print("Creating joystick listener...")
    # joystick_interface = JoystickInterface(config)
    socket_Pi = SocketInterface_Pi(HOST_IP="192.168.31.88", HOST_PORT=50000)
    # print("Done.")

    last_loop = time.time()

    # print("Summary of gait parameters:")
    # print("overlap time: ", config.overlap_time)
    # print("swing time: ", config.swing_time)
    # print("z clearance: ", config.z_clearance)
    # print("x shift: ", config.x_shift)

    # Wait until the activate button has been pressed

    print("Waiting for L1 to activate robot.")
    while True:
        # command = joystick_interface.get_command(state)
        # print(command.activate_event)
        # joystick_interface.set_color(config.ps4_deactivated_color)

        msg_recv = socket_Pi.subscriber_cmd()
        command = socket_Pi.get_command(state, msg_recv)

        if command.activate_event == 1:
            break
        print("You should firstly activate the Robot!")
        time.sleep(0.1)
    print("Robot activated.")
    # joystick_interface.set_color(config.ps4_color)

    while True:
        now = time.time()
        if now - last_loop < config.dt:
            continue
        last_loop = time.time()

        # Parse the udp joystick commands and then update the robot controller's parameters
        
        print("Waiting for msg...")
        msg_recv = socket_Pi.subscriber_cmd()
        print("Pi receive:", msg_recv)
        command = socket_Pi.get_command(state, msg_recv)

        if command.activate_event == 1:
            print("Deactivating Robot")
            break

        # Read imu data. Orientation will be None if no data was available
        quat_orientation = (
            imu.read_orientation() if use_imu else np.array([1, 0, 0, 0])
        )
        state.quat_orientation = quat_orientation

        # Step the controller forward by dt
        controller.run(state, command)

        # Update the pwm widths going to the servos
        hardware_interface.set_actuator_postions(state.joint_angles)

In [30]:
import os
os.system("sudo pigpiod")
robot()

2023-06-15 17:09:43 initInitialise: Can't lock /var/run/pigpio.pid
Can't initialise pigpio library


Starting socket: TCP...
TCP server listen @ 192.168.31.88:50000!
Connection accepted from 192.168.31.229.
Waiting for L1 to activate robot.
Robot activated.
Waiting for msg...
Pi receive: {'ly': 0.0, 'lx': 0.0, 'rx': 0.0, 'ry': 0.0, 'R1': 0, 'L1': 1, 'dpady': 0.0, 'dpadx': 0.0, 'x': 0, 'message_rate': 50}
Waiting for msg...
Pi receive: {'ly': 0.0, 'lx': 0.0, 'rx': 0.0, 'ry': 0.0, 'R1': 0, 'L1': 1, 'dpady': 0.0, 'dpadx': 0.0, 'x': 0, 'message_rate': 50}
Waiting for msg...


KeyboardInterrupt: 